In [2]:
import sqlite3
import pandas as pd
import numpy as np
import sqlglot
import json
import time
dbList = {
    "california_schools": "D:/NL2SQLDev/backend/data/dbCollection/california_schools/california_schools.sqlite",
    "card_games": "D:/NL2SQLDev/backend/data/dbCollection/card_games/card_games.sqlite",
    "codebase_community": "D:/NL2SQLDev/backend/data/dbCollection/codebase_community/codebase_community.sqlite",
    "debit_card_specializing": "D:/NL2SQLDev/backend/data/dbCollection/debit_card_specializing/debit_card_specializing.sqlite",
    "european_football_2": "D:/NL2SQLDev/backend/data/dbCollection/european_football_2/european_football_2.sqlite",
    "financial": "D:/NL2SQLDev/backend/data/dbCollection/financial/financial.sqlite",
    "formula_1": "D:/NL2SQLDev/backend/data/dbCollection/formula_1/formula_1.sqlite",
    "student_club": "D:/NL2SQLDev/backend/data/dbCollection/student_club/student_club.sqlite",
    "superhero": "D:/NL2SQLDev/backend/data/dbCollection/superhero/superhero.sqlite",
    "thrombosis_prediction": "D:/NL2SQLDev/backend/data/dbCollection/thrombosis_prediction/thrombosis_prediction.sqlite",
    "toxicology": "D:/NL2SQLDev/backend/data/dbCollection/toxicology/toxicology.sqlite",
}

In [50]:
task = "card_games"

In [51]:
con=sqlite3.connect(dbList[task])
cursor=con.cursor()

In [52]:
def get_table_names(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [table[0] for table in tables]


def executeSQL(cursor,sql):
    cursor.execute(sql)
    rows = cursor.fetchall()
    columns = [i[0] for i in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    return df
def generate_schema( db_path, num_rows=None):
    full_schema_prompt_list = []
    conn = sqlite3.connect(db_path)
    # Create a cursor object
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    schemas = {}
    for table in tables:
        if table == "sqlite_sequence":
            continue
        cursor.execute(f"PRAGMA table_info('{table[0]}')")
        columns_info = cursor.fetchall()
        column_df = pd.DataFrame(
            columns_info,
            columns=["cid", "name", "type", "notnull", "dflt_value", "pk"],
        )
        cursor.execute(f"PRAGMA foreign_key_list('{table[0]}')")
        fk_info = cursor.fetchall()
        fk_df = pd.DataFrame(
            fk_info,
            columns=[
                "id",
                "seq",
                "table",
                "from",
                "to",
                "on_update",
                "on_delete",
                "match",
            ],
        )
        schemas[table[0]] = {"columns": column_df, "foreign_keys": fk_df}
    return schemas

def generate_fk_pairs_list( schema):
    fk_pairs_list = []
    for k, v in schema.items():
        for iter, row in v["foreign_keys"].iterrows():
            pair_1 = (k, row["from"])
            pair_2 = (row["table"], row["to"])
            flag = 0
            for j in range(len(fk_pairs_list)):
                if pair_1 in fk_pairs_list[j] or pair_2 in fk_pairs_list[j]:
                    fk_pairs_list[j].add(pair_1)
                    fk_pairs_list[j].add(pair_2)
                    flag = 1
                    break
            if flag == 0:
                fk_pairs_list.append(set([pair_1, pair_2]))
    return fk_pairs_list
def jsonFKPairs( fk_pairs):
    fk_json_list = []
    for fk_pair in fk_pairs:
        fk_json_list.append([])
        for fk in fk_pair:
            fk_json_list[-1].append({"table": fk[0], "column": fk[1]})
    return fk_json_list
def generateFkpPairsList( schema):
    fk_pairs_list = []
    for k, v in schema.items():
        for iter, row in v["foreign_keys"].iterrows():
            pair_1 = (k, row["from"])
            pair_2 = (row["table"], row["to"])
            flag = 0
            for j in range(len(fk_pairs_list)):
                if pair_1 in fk_pairs_list[j] or pair_2 in fk_pairs_list[j]:
                    fk_pairs_list[j].add(pair_1)
                    fk_pairs_list[j].add(pair_2)
                    flag = 1
                    break
            if flag == 0:
                fk_pairs_list.append(set([pair_1, pair_2]))
    return fk_pairs_list


def schemaTrans( schema):
    result = {}
    result["table_names"] = []
    result["columns"] = {}
    result["fk_pairs"] = []
    for k, v in schema.items():
        result["table_names"].append(k)
        result["columns"][k] = v["columns"].to_dict()
    fk_pairs = generateFkpPairsList(schema)
    result["fk_pairs"] = jsonFKPairs(fk_pairs)
    return result

def dbSchemaPrompt( dbName=None, dbSchema=None):

    prompt = f"The database schema of {dbName} is as follows:\n"
    for i in dbSchema["columns"]:
        prompt += f"    Table {i}: ("
        for j in dbSchema["columns"][i]["name"]:
            prompt += f"'{dbSchema['columns'][i]['name'][j]}', "
        prompt = prompt[:-2] + ")\n"
    prompt += "The foreign key pairs are:\n"
    for i in dbSchema["fk_pairs"]:
        for j in i:
            prompt += f"{j['table']}.'{j['column']}' = "
        prompt = prompt[:-2] + "\n"
    return prompt

In [53]:
scheam_original=generate_schema(dbList[task])
schema=schemaTrans(scheam_original)
#pd.DataFrame(schema["columns"]["schools"])["name"].tolist()

In [54]:
schema['fk_pairs']

[[{'table': 'cards', 'column': 'uuid'},
  {'table': 'rulings', 'column': 'uuid'},
  {'table': 'legalities', 'column': 'uuid'},
  {'table': 'foreign_data', 'column': 'uuid'}],
 [{'table': 'sets', 'column': 'code'},
  {'table': 'set_translations', 'column': 'setCode'}]]

In [55]:
# read json files
with open("D:/NL2SQLDev/task_card_games.json", "r") as f:
    sol = json.load(f)
with open("D:/NL2SQLDev/task_gaosh1_sql_card_games.json", "r") as f:
    data = json.load(f)

In [56]:
executeSQL(cursor, sol[0]["sql"])

,name
0,Ballista Squad
1,Bandage
2,Beacon of Immortality
3,"Cho-Manno, Revolutionary"
4,Condemn
...,...
25056,Into the Roil
25057,Bloodchief's Thirst
25058,Roil Eruption
25059,Roiling Regrowth


In [57]:
executeSQL(cursor, data[0]["sql"])

,name
0,Ballista Squad
1,Bandage
2,Beacon of Immortality
3,"Cho-Manno, Revolutionary"
4,Condemn
...,...
25056,Into the Roil
25057,Bloodchief's Thirst
25058,Roil Eruption
25059,Roiling Regrowth


In [58]:
executeSQL(cursor, sol[1]["sql"])

,id
0,23004
1,25350
2,25355
3,25357
4,25359
...,...
67,56710
68,56712
69,56714
70,56716


In [59]:
executeSQL(cursor, data[1]["sql"])

,name,id
0,Void Beckoner,23004
1,"Valki, God of Lies // Tibalt, Cosmic Impostor",25350
2,Barkchannel Pathway // Tidechannel Pathway,25355
3,Blightstep Pathway // Searstep Pathway,25357
4,Darkbore Pathway // Slitherbore Pathway,25359
...,...,...
67,Brightclimb Pathway // Grimclimb Pathway,56710
68,Clearwater Pathway // Murkwater Pathway,56712
69,Cragcrown Pathway // Timbercrown Pathway,56714
70,Needleverge Pathway // Pillarverge Pathway,56716


In [60]:
executeSQL(cursor, sol[2]["sql"])

,id
0,17983
1,18058
2,29523
3,38736
4,38737


In [61]:
executeSQL(cursor, data[2]["sql"])

,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,17983,Yongjae Choi,None,"arena,mtgo,paper",black,227736,227334,"G,U",None,"G,U",...,Oko,Legendary,198356,[+2]: Create a Food token. (It's an artifact w...,None,Legendary Planeswalker — Oko,Planeswalker,46153afe-5e05-5082-852a-648c03924bcf,f203bad8-9c07-507c-9699-fc8fec69e2d2,None
1,18058,Wesley Burt,None,"mtgo,paper",borderless,228008,227483,"G,U",None,"G,U",...,Oko,Legendary,198357,[+2]: Create a Food token. (It's an artifact w...,None,Legendary Planeswalker — Oko,Planeswalker,f203bad8-9c07-507c-9699-fc8fec69e2d2,46153afe-5e05-5082-852a-648c03924bcf,None
2,29523,Mike Bierek,None,"arena,mtgo,paper",black,219702,None,U,None,U,...,None,None,169146,Take an extra turn after this one.\nIf Nexus o...,None,Instant,Instant,f2b2679f-0d84-5602-8014-241725c94023,None,None
3,38736,Yongjae Choi,None,paper,black,228295,228222,"G,U",None,"G,U",...,Oko,Legendary,200395,[+2]: Create a Food token. (It's an artifact w...,None,Legendary Planeswalker — Oko,Planeswalker,3d8884e6-75a2-5422-98e0-44b155f71db7,8c48df5c-6e6f-5602-8312-523435d88a9b,None
4,38737,Yongjae Choi,None,paper,black,228154,None,"G,U",None,"G,U",...,Oko,Legendary,199928,[+2]: Create a Food token. (It's an artifact w...,None,Legendary Planeswalker — Oko,Planeswalker,8c48df5c-6e6f-5602-8312-523435d88a9b,3d8884e6-75a2-5422-98e0-44b155f71db7,None


In [62]:
executeSQL(cursor, sol[3]["sql"])

,status
0,Legal
1,Restricted
2,Banned


In [63]:
executeSQL(cursor, data[3]["sql"])

,status
0,Legal
1,Restricted
2,Banned


In [64]:
executeSQL(cursor, sol[4]["sql"])

,CAST(SUM(CASE WHEN T2.language = 'French' THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(T1.id)
0,12.977106


In [65]:
executeSQL(cursor, sol[4]["sql"])

,CAST(SUM(CASE WHEN T2.language = 'French' THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(T1.id)
0,12.977106


In [66]:
executeSQL(cursor, sol[5]["sql"])

,name,id
0,Commander's Arsenal,50
1,Premium Deck Series: Fire and Lightning,307
2,From the Vault: Exiled,522
3,From the Vault: Relics,523
4,From the Vault: Legends,524
5,From the Vault: Realms,525
6,From the Vault: Twenty,526
7,From the Vault: Annihilation,527
8,From the Vault: Angels,528
9,From the Vault: Lore,529


In [67]:
executeSQL(cursor, data[5]["sql"])

,id
0,50
1,307
2,522
3,523
4,524
5,525
6,526
7,527
8,528
9,529


In [68]:
executeSQL(cursor, sol[6]["sql"])

,name
0,Rags // Riches
1,Rags // Riches
2,"Elbrus, the Binding Blade // Withengar Unbound"
3,Realm-Cloaked Giant // Cast Off
4,Reaper of Night // Harvest Fear
5,Beanstalk Giant // Fertile Footsteps
6,Flaxen Intruder // Welcome Home
7,Realm-Cloaked Giant // Cast Off
8,Reaper of Night // Harvest Fear
9,Beanstalk Giant // Fertile Footsteps


In [69]:
executeSQL(cursor, data[6]["sql"])

,name,faceConvertedManaCost
0,Rags // Riches,7.0
1,Rags // Riches,7.0
2,"Elbrus, the Binding Blade // Withengar Unbound",7.0
3,Realm-Cloaked Giant // Cast Off,7.0
4,Reaper of Night // Harvest Fear,7.0
5,Beanstalk Giant // Fertile Footsteps,7.0
6,Flaxen Intruder // Welcome Home,7.0
7,Realm-Cloaked Giant // Cast Off,7.0
8,Reaper of Night // Harvest Fear,7.0
9,Beanstalk Giant // Fertile Footsteps,7.0


In [70]:
executeSQL(cursor, sol[7]["sql"])

,id
0,835
1,854
2,857
3,927
4,988
...,...
649,56366
650,56367
651,56378
652,56384


In [71]:
executeSQL(cursor, data[7]["sql"])

,name,id
0,Path to Exile,835
1,Brainstorm,854
2,Cyclonic Rift,857
3,Blasphemous Act,927
4,Reclamation Sage,988
...,...,...
649,Arid Mesa,56366
650,Misty Rainforest,56367
651,Ancient Tomb,56378
652,Prismatic Vista,56384


In [72]:
executeSQL(cursor, sol[8]["sql"])

,id,artist
0,5,Rebecca Guay
1,6,Rob Alexander
2,10,Daren Bader
3,11,Thomas M. Baxa
4,13,Christopher Rush
...,...,...
29931,56825,Adam Paquette
29932,56828,Campbell White
29933,56829,Jason Rainville
29934,56830,Campbell White


In [73]:
executeSQL(cursor, data[8]["sql"])

,id,artist
0,5,Rebecca Guay
1,6,Rob Alexander
2,10,Daren Bader
3,11,Thomas M. Baxa
4,13,Christopher Rush
...,...,...
29931,56825,Adam Paquette
29932,56828,Campbell White
29933,56829,Jason Rainville
29934,56830,Campbell White


In [74]:
executeSQL(cursor, sol[9]["sql"])

,id,text,hasContentWarning
0,29,You can’t put an Aura card from your hand onto...,0
1,3872,"This card now has Enchant Swamp, which works e...",0
2,6651,Cycling is an activated ability. Effects that ...,0
3,6652,Cycling is an activated ability. Effects that ...,0
4,7950,Cycling is an activated ability. Effects that ...,0
5,7970,Cycling is an activated ability. Effects that ...,0
6,8107,Cycling is an activated ability. Effects that ...,0
7,8109,Cycling is an activated ability. Effects that ...,0
8,8448,Cycling is an activated ability. Effects that ...,0
9,8787,Cycling is an activated ability. Effects that ...,0


In [75]:
executeSQL(cursor, data[9]["sql"])

,id,text
0,29,You can’t put an Aura card from your hand onto...
1,3872,"This card now has Enchant Swamp, which works e..."
2,6651,Cycling is an activated ability. Effects that ...
3,6652,Cycling is an activated ability. Effects that ...
4,7950,Cycling is an activated ability. Effects that ...
5,7970,Cycling is an activated ability. Effects that ...
6,8107,Cycling is an activated ability. Effects that ...
7,8109,Cycling is an activated ability. Effects that ...
8,8448,Cycling is an activated ability. Effects that ...
9,8787,Cycling is an activated ability. Effects that ...


In [76]:
executeSQL(cursor, sol[10]["sql"])

,CAST(SUM(CASE WHEN T2.language = 'French' THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(T1.id)
0,10.0


In [77]:
executeSQL(cursor, data[10]["sql"])

,percentage_french_cards
0,0.1


In [78]:
executeSQL(cursor, sol[11]["sql"])

,CAST(SUM(CASE WHEN T1.hasContentWarning = 0 THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(T1.id)
0,100.0


In [79]:
executeSQL(cursor, data[11]["sql"])

,percentage
0,1.0


In [25]:
def time_minus(h1,a1,b1,h2,a2,b2):
    return (h1-h2)*60*60+(a1-a2)*60+(b1-b2)
a= 'Usr2_7_2'
with open(f"D:/NL2SQLDev/backend/data/log/{a}.json", "r") as f:
    log_record = json.load(f)
t1 = log_record["createTime"]
t2 = log_record["endTime"]
t1=time.strptime(t1, "%Y/%m/%d %H:%M:%S")
t2=time.strptime(t2, "%Y/%m/%d %H:%M:%S")
print(time_minus(t2.tm_hour,t2.tm_min, t2.tm_sec, t1.tm_hour,t1.tm_min, t1.tm_sec)) 

216


In [1]:
def time_minus_1( a1, b1,  a2, b2):
    return + (a1 - a2) * 60 + (b1 - b2)
def time_add(t,a1,a2,a3):
    time_all = t+a1*60*60+a2*60+a3
    th=time_all//3600
    time_all=time_all%3600
    tm=time_all//60
    time_all=time_all%60
    ts=time_all
    return th,tm,ts

In [82]:
time_minus_1(45,51,47,5)

-74